# ◈ Setting Up Dependencies

### AIM: To install, import and connect the required python libraries & APIs 

In [ ]:
# Installing Tensorflow to build a LSTM neural network

!pip install tensorflow

In [ ]:
# Installing openCV to access webcam for data collection and predictions

!pip install opencv-python

In [ ]:
# Connecting to mediapipe API for landmarks

!pip install mediapipe

In [ ]:
# APPLYING NECESSARY IMPORTS

import cv2
import numpy as np
import matplotlib.pyplot as plt
import time
import mediapipe as mp
import os

# ◈ Accessing Keypoints

### AIM: To obtain the information about facial expressions and bodily gestures using Mediapipe API

In [ ]:
# CREATING OBJECT OF THE HOLISTIC MODEL & 

mp_holistic = mp.solutions.holistic         #Holistic Model
mp_drawing = mp.solutions.drawing_utils     #Drawing utilities

In [ ]:
def mideapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)     #BGR to RGB color conversion
    image.flags.writeable = False                      #Image not writeable
    results = model.process(image)                     #Using mediapipe to make detections
    image.flags.writeable = True                       #Image writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)     #RGB to BGR color conversion
    return image, results

In [ ]:
# THIS FUNCTION WAS THE KEY TO GET NUMERIC DATA FROM THE IMAGES ACCESSED USING OpenCV

def draw_styled_landmarks(image, results):
    #Draw Face Connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(color = (80, 110, 10), thickness = 1, circle_radius = 1),
                              mp_drawing.DrawingSpec(color = (80, 256, 121), thickness = 1, circle_radius = 1)
                             )   
    #Draw Pose Connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color = (80, 22, 10), thickness = 2, circle_radius = 2),
                              mp_drawing.DrawingSpec(color = (80, 44, 121), thickness = 2, circle_radius = 1)
                             ) 
    #Draw Left Hand Connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color = (121, 22, 76), thickness = 2, circle_radius = 2),
                              mp_drawing.DrawingSpec(color = (121, 44, 250), thickness = 2, circle_radius = 1)
                             )  
    #Draw Right Hand Connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color = (121, 22, 76), thickness = 2, circle_radius = 2),
                              mp_drawing.DrawingSpec(color = (121, 44, 250), thickness = 2, circle_radius = 1)
                             )    

#### The code snippet below was used as a trial to see the landmarks and features being marked in real time

In [ ]:
cap = cv2.VideoCapture(0)   
                        #0 works for window webcams, you can try 1 & 2 according to the OS and webcam type (External or Internal)

#Using mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
    
        #Read Feed
        ret, frame = cap.read()
    
        #Make Detections
        image, results = mideapipe_detection(frame, holistic)
        print(results)

        #Draw Landmarks
        draw_styled_landmarks(image, results)
    
        #Show to Screen
        cv2.imshow('OpenCV Feed', image)
    
        #Break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# FORCEFULLY RELEASING THE WEBCAM

cap.release()
cv2.destroyAllWindows()

# ◈ Keypoint Extraction

### AIM: Extracting the landmarks processed earlier into useable format

In [ ]:
# FUNCTION BELOW ACCESSES THE COORDINATES OF THE LANDMARKS AND TURNS THEM INTO A 1 DIMENSIONAL NUMPY ARRAY

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    return np.concatenate([pose, face, lh, rh])

# ◈ Directory Setup

### AIM: Connecting of the internal file directories so as to create a new folder for data collection and storage

In [ ]:
# CREATING A PIPELINE TO THE LOCAL DIRECTORY & ADDING DATA SPECIFICATIONS

# Using os module in python for easy access of the local directories and wide range usability
DATA_PATH = os.path.join('MP_Data')

# Specifying the actions to be accomodated in the project & defining its scope
actions = np.array(['hello', 'thanks', 'iloveyou'])

# It is the number of videos (sequence) to be collected for each action
no_sequences = 30

# This defines the number of frames each video (sequence) will have
sequence_length = 30

In [ ]:
# CREATING A COMPLETE SUB-DIRECTORY TREE WITHIN THE DIRECTORY

for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

#### Data strorage directory named 'MP_Data's structure (after running the above cell)

```
MP_Data/
│
├── hello/
│   ├── 0/
│   ├── 1/
│   ├── 2/
│   ├── ...
│   ├── 29/
│
├── thanks/
│   ├── 0/
│   ├── 1/
│   ├── 2/
│   ├── ...
│   ├── 29/
│
├── iloveyou/
│   ├── 0/
│   ├── 1/
│   ├── 2/
│   ├── ...
│   ├── 29/
```

# ◈ Data Collection

### AIM: Collecting keypoint values for training & testing and thier storage

In [ ]:
# USED TO COLLECT DATA (VIDEOS OR SEQUENCES) WITHIN THE ABOVE CREATE FOLDERS

cap = cv2.VideoCapture(0)

#Using mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:

    #Loop through actions
    for action in actions:
        #Loop through sequences aka videos
        for sequence in range(no_sequences):
            #Loop through video length aka frame
            for frame_num in range(sequence_length):
                
                #Read Feed
                ret, frame = cap.read()
            
                #Make Detections
                image, results = mideapipe_detection(frame, holistic)
                print(results)
        
                #Draw Landmarks
                draw_styled_landmarks(image, results)

                #Apply collection logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING_COLLECTION', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(3000)
                else:
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                #Export Keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
            
                #Show to Screen
                cv2.imshow('OpenCV Feed', image)
            
                #Break
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    #Releases the webcam automatically as soon as all the directories are visited for successful data storage
    cap.release()
    cv2.destroyAllWindows()

#### Directory structure after data collection & storage

```
MP_Data/
│
├── hello/
│   ├── 0/
│   │   ├── 0.npy
│   │   ├── 1.npy
│   │   ├── ...
│   │   ├── 29.npy
│   ├── 1/
│   ├── 2/
│   ├── ...
│   ├── 29/
│
├── thanks/
│   ├── 0/
│   │   ├── 0.npy
│   │   ├── 1.npy
│   │   ├── ...
│   │   ├── 29.npy
│   ├── 1/
│   ├── 2/
│   ├── ...
│   ├── 29/
│
├── iloveyou/
│   ├── 0/
│   │   ├── 0.npy
│   │   ├── 1.npy
│   │   ├── ...
│   │   ├── 29.npy
│   ├── 1/
│   ├── 2/
│   ├── ...
│   ├── 29/
```

# ◈ Data Preprocessing & Label + Feature Creation

### AIM: To preprocess & analyze the data and making it training and testing worthy

In [ ]:
from sklearn.model_selection import train_test_split    #To split the data into training and testing sets
from tensorflow.keras.utils import to_categorical       #One-Hot Ended formatting to labels (classes)

In [ ]:
# CREATING A LABEL MAP OF THE PRE-DEFINED ACTIONS

label_map = {label:num for num, label in enumerate(actions)}
label_map

In [ ]:
# LOADING IN THE COLLECTED DATA

sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

#### Basic Analysis

In [ ]:
np.array(sequences).shape

In [ ]:
np.array(labels).shape

#### Splitting data & analyzing the partitions

In [ ]:
X = np.array(sequences)

In [ ]:
y = to_categorical(labels).astype(int)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
y_train.shape, y_test.shape

# ◈ Model Building & Training

### AIM: Using tensorflow & keras for model building and training

In [ ]:
from tensorflow.keras.models import Sequential          #Leveraging the sequential model under keras
from tensorflow.keras.layers import LSTM, Dense         #Importing the layers to be used in the model
from tensorflow.keras.callbacks import TensorBoard      #Importing tensorboard for easy visualizations of model's metrics
actions = np.array(['hello', 'thanks', 'iloveyou'])

In [ ]:
# SETTING UP THE 'Logs' DIRECTORY FOR TENSORBOARD INITIALISATION

log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir = log_dir)

#### Steps to access TensorBoard

1.**Open Command Prompt in the Logs Directory**  
-Navigate to the directory where your TensorBoard logs are stored.  
-If you are inside Jupyter Notebook, you can find the logs directory using:  
```python
import os
print(os.getcwd())  # This prints the current working directory
```  
-Open **Command Prompt (cmd)** and navigate to the logs folder using:  
```
cd path\to\logs\directory
```  

2.**Run TensorBoard**  
-In the command prompt, type the following command:  
```
tensorboard --logdir=logs --port=6006
```  
-Replace `logs` with the actual path to your log directory if needed.  

3.**Access TensorBoard in Browser**  
-After running the command, TensorBoard will start and provide a URL like:  
```
http://localhost:6006/
```  
-Open this URL in your web browser to view TensorBoard.  

In [ ]:
# A NEURAL NETWORK IS BUILD IN THE SEQUENTIAL MODEL WITH LSTM & DENSE LAYERS HAVING APPROPRIATE NEURONS

model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = 'relu', input_shape = (30, 1662)))
model.add(LSTM(128, return_sequences = True, activation = 'relu'))
model.add(LSTM(64, return_sequences = False, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(actions.shape[0], activation = 'softmax'))

In [ ]:
# MODEL COMPILATION IS CARRIED OUT

model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy'])

In [ ]:
# MODEL IS FIT OVER THE TRAINING DATASETS (SUPERVISED LEARNING AS OUTPUT LABELS WILL ALSO BE PROVIDED)

model.fit(X_train, y_train, epochs = 1000, callbacks = [tb_callback])   
                            # NOTE: Stop the epochs as soon as the desired accuracy is achieved to avoid overfit

In [ ]:
# A GENERAL SUMMARISATION OF MODEL AND ITS PARAMETERS

model.summary()

# ◈ Save Weights

### AIM: Saving the model & its parameters for future use

In [ ]:
#Saving the model
model.save('action.h5')

In [ ]:
# #Deletion of model
# del model

In [ ]:
#Loading weights into a newly created neural network and bypassing the time used for fitting it over training data
model.load_weights('action.h5')

# ◈ Test Predictions

### AIM: Making predictions over testing dataset

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# ◈ Model Evaluation

### AIM: Evaluating model using confusion matrix & accuracy score

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis = 1).tolist()
yhat = np.argmax(yhat, axis = 1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)    #Visualising the apt predictions of True Negatives and True Positives

In [ ]:
accuracy_score(ytrue, yhat)     #Gaining a metrics of the model predictions in comparison to the actual values

# ◈ Real Time Detection

### AIM: Achieving the final goal by making real time detections

In [ ]:
# WE CAN START MAKING REAL TIME PREDICTIONS BY HAVING THE WEBCAM CAPTURE OUR GESTURES
        #THROUGH THE FUNCTION BELOW, WE CAN SEE THE PREDICTIONS (INTERPRETATIONS OF SIGN LANGUAGE) ON THE TOP OF THE SCREEN

#Detection Variables
sequence = []
sentence = []
threshold = 0.995  #Can be tuned according to the requirement

cap = cv2.VideoCapture(0)

#Using mediapipe model
with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
    while cap.isOpened():
    
        #Read Feed
        ret, frame = cap.read()
    
        #Make Detections
        image, results = mideapipe_detection(frame, holistic)
        print(results)

        #Draw Landmarks
        draw_styled_landmarks(image, results)

        #Prediction Logic
        keypoints = extract_keypoints(results)
        sequence.insert(0, keypoints)
        sequence = sequence[:30]

        thres = res[np.argmax(res)]
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis = 0))[0]
            print(thres)
            
        #Visualization Logic
        if thres > threshold:
            if len(sentence) > 0:
                if actions[np.argmax(res)] != sentence[-1]:
                    sentence.append(actions[np.argmax(res)])
            else:
                sentence.append(actions[np.argmax(res)])

        if len(sentence) > 5:
            sentence = sentence[-5:]

        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
        #Show to Screen
        cv2.imshow('OpenCV Feed', image)
    
        #Break
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
#RELEASING THE WEB CAM AFTER SUCCESSFUL INTERPRETATIONS & SEAMLESS COMMUNICATION

cap.release()
cv2.destroyAllWindows()